# Toronto Data Collection

This notebook will be used to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

Additional Instructions are below:

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

Note: There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use pandas to read the table into a pandas dataframe.

Another way, which would help to learn for more complicated cases of web scraping is using the BeautifulSoup package. Here is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/

Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe.

## Step 1 - Scraping data


Import `pandas` to create the dataframe, `urlopen` to make a request and `BeautifulSoup` to read through the HTML page.

In [4]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

Load the link and fetch the table from the HTML. The first table on the page is the one we will be using.

In [5]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(r.text, 'html.parser')
table=soup.find('table', attrs={'class':'wikitable sortable'})
#print(table.prettify())

Next, iterate through all rows in the table except the first (which is the header) and save the contents to the variable data. Then create the Pandas dataframe.

In [6]:
data = []
for tr in table.find_all('tr')[1:]:
    row_data = tr.find_all('td')
    data.append([cell.text for cell in row_data])
dataset = pd.DataFrame(data, columns = ['PostalCode', 'Borough', 'Neighborhood'])
dataset.head(5)

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


## Cleaning the data



Remove all the unnecessary rows where Borough  is Not assigned

In [7]:
dataset = dataset[dataset['Borough'] != 'Not assigned\n']
dataset.reset_index(drop = True, inplace = True)
dataset.head(5)

,PostalCode,Borough,Neighborhood
0,M3A\n,North York\n,Parkwoods\n
1,M4A\n,North York\n,Victoria Village\n
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n"
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n"


Assign the Neighbourhood to same as Borough if it is Not assigned.  If the data in the column Neighbourhood  has \n at the end of each name, remove it first befire the assignment.

In [8]:
dataset['Neighborhood'] = dataset['Neighborhood'].str.split('\n', expand = True)[0]
dataset.loc[dataset['Neighborhood'] == 'Not assigned', 'Neighborhood'] = dataset.loc[dataset['Neighborhood'] == 'Not assigned', 'Borough']
dataset.head(5)

,PostalCode,Borough,Neighborhood
0,M3A\n,North York\n,Parkwoods
1,M4A\n,North York\n,Victoria Village
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights"
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government"


Combine the rows with the same PostCode and Borough.  If the data in the  column PostalCode and Borough has a \n at the end of each name, remove it.

In [9]:
dataset['Borough'] = dataset['Borough'].str.split('\n', expand = True)[0]
dataset['PostalCode'] = dataset['PostalCode'].str.split('\n', expand = True)[0]
dataset = dataset.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
dataset.head(5)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Saving the Data


In [10]:
dataset.to_csv('dataset.csv', index = False)

In [11]:
print("Dataset shape: {}".format(dataset.shape))

Dataset shape: (103, 3)
